<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/%ED%86%B5%ED%95%A9/(%EC%9C%B5%ED%95%A91)_NMSC_%EC%97%AD%EB%B2%88%EC%97%AD_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

다음 코드는 BACK-TRANSLATION을 사용해서 데이터를 증분하는 방법입니다.
- 증분대상 데이터(0 라벨 데이터)를 80% 데이터셋에서는 1000개, 50%는 2500개 전체, 10%는 500개 전체, 2%는 100개 전체를 파일로 저장
- "(2)NSMC_train_불균형_0.02_증분대상 100개.csv" 이런식으로 저장해 줌

In [1]:
pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.8 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=debc7d0d5662cdd4ce41793a16713e386061965b7dee844450d3604053ba460f
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfram

In [2]:
import pandas as pd
from googletrans import Translator

def back_translate(text, source_lang, pivot_lang):
  try:
    translator = Translator()
    translated_to_pivot = translator.translate(text, src=source_lang, dest=pivot_lang).text
    back_translated = translator.translate(translated_to_pivot, src=pivot_lang, dest=source_lang).text
    return back_translated
  except:
    return text


def augment_data(texts, source_lang, pivot_langs, multiplier):
    augmented_texts = []
    for text in texts:
        for _ in range(multiplier - 1):
            for pivot_lang in pivot_langs:
                augmented_texts.append(back_translate(text, source_lang, pivot_lang))
    return augmented_texts

def augment_dataframe(df, text_column, category_column, target_category, source_lang, pivot_langs, multiplier):
    df_augmented = df.copy()
    target_texts = df[df[category_column] == target_category][text_column].tolist()
    augmented_texts = augment_data(target_texts, source_lang, pivot_langs, multiplier)
    augmented_rows = [{text_column: text, category_column: target_category} for text in augmented_texts]
    df_augmented = pd.concat([df_augmented, pd.DataFrame(augmented_rows)], ignore_index=True)
    return df_augmented

# 데이터 증분하기(2% 데이터셋)

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'   이전   졸업논문


In [4]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/NSMC2_0.02_증분대상_100개.csv', encoding='utf-8-sig')

In [5]:
#'en', 'ja', 'zh-cn', 'zh-tw', 'fr', 'de', 'es', 'ru', 'pt', 'hi'
#원데이터 + multiplier-1 * pivot_languages 수
pivot_languages = ['en', 'ja', 'zh-cn'] #20개 언어

In [6]:
len(pivot_languages)

3

In [7]:
multiplier=2
df_2_augmented = augment_dataframe(df, text_column='document', category_column='label', target_category=0, source_lang='ko', pivot_langs=pivot_languages, multiplier=multiplier)

In [8]:
df_2_augmented.to_csv('/content/gdrive/MyDrive/졸업논문/NSMC2_불균형_0.02_aug_bt_300개.csv', encoding='utf-8-sig')